In [0]:
#@title
from google.colab import drive
drive.mount('/content/drive')

# TEST_DIR = '/content/drive/My Drive/brain training/'
TEST_DIR = '/content/drive/My Drive/Alzheimer\'s Research/brain training/Results/GreyWhiteMasks/'
TRUTH_DIR = '/content/drive/My Drive/Alzheimer\'s Research/Ground Truth Accuracy Calculation/ground_truth_images/'
SAVE_FILENAME = 'AccuracyResults.csv' # Under TEST_DIR
SAVE_FILENAME_LATEX = 'AccuracyResults.txt' # Under TEST_DIR

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
#@title
import os
import glob
import numpy as np
from tqdm import tqdm
import re
from PIL import Image
Image.MAX_IMAGE_PIXELS = None

In [0]:
#@title
# Stores expected image dimensions
IMAGEDIMNORM = np.array(
        [[55296, 47616], 
        [73728, 47616], 
        [66048, 46080], 
        [69120, 36864], 
        [59904, 43008], 
        [56832, 39936], 
        [62976, 47616], 
        [58368, 44544], 
        [50688, 47616], 
        [58368, 47616], 
        [55296, 46080], 
        [72192, 47616], 
        [69120, 47616], 
        [49152, 43008], 
        [50688, 36864], 
        [55296, 46080], 
        [55296, 41472], 
        [56832, 43008]], dtype='int')
downscale = 128
IMAGEDIMORIG = np.array(
        [[53784, 47177], 
        [73704, 46200], 
        [65736, 45178], 
        [67728, 36459], 
        [59760, 41787], 
        [55776, 39849], 
        [61752, 46410], 
        [57768, 43740], 
        [49800, 47520], 
        [57768, 47481], 
        [53784, 45142], 
        [71712, 47426], 
        [67728, 47200], 
        [47808, 41668], 
        [49800, 36131], 
        [53784, 45548], 
        [53784, 41073], 
        [55776, 41634]], dtype='int')

In [0]:
#@title
# Get image names
image_names = glob.glob(TRUTH_DIR + "*Grey.png")
image_names = [imagename.split('/')[-1] for imagename in image_names]
image_names = [imagename.split('-Grey')[0] for imagename in image_names]
image_names = sorted(image_names)
print(image_names)

['NA3777-02_AB', 'NA4077-02_AB', 'NA4092-02_AB', 'NA4107-02_AB', 'NA4160-02_AB', 'NA4195-02_AB', 'NA4256-02_AB', 'NA4299-02_AB', 'NA4391-02_AB', 'NA4450-02_AB', 'NA4463-02_AB', 'NA4471-02_AB', 'NA4553-02_AB', 'NA4626-02_AB', 'NA4672-02_AB', 'NA4675-02_AB', 'NA4691-02_AB', 'NA4695-02_AB']


#### Generate ground-truth background masks

In [0]:
#@title
def img_frombytes(data):
  size = data.shape[::-1]
  databytes = np.packbits(data, axis=1)
  return Image.frombytes(mode='1', size=size, data=databytes)

t = tqdm(total=len(image_names))
for i, imagename in enumerate(image_names):
  # Filename for testing and ground truth images
  truth_grey_img_name = imagename + "-Grey.png"
  truth_white_img_name = imagename + "-White.png"

  # Load images
  truth_grey_arr = np.array(Image.open(TRUTH_DIR + truth_grey_img_name))
  truth_white_arr = np.array(Image.open(TRUTH_DIR + truth_white_img_name))
  assert(truth_grey_arr.shape[0] == IMAGEDIMORIG[i, 1] and truth_grey_arr.shape[1] == IMAGEDIMORIG[i, 0])
  assert(truth_white_arr.shape[0] == IMAGEDIMORIG[i, 1] and truth_white_arr.shape[1] == IMAGEDIMORIG[i, 0])
  assert(truth_grey_arr.dtype == np.bool)
  assert(truth_white_arr.dtype == np.bool)

  # Update tqdm descriptions
  t.set_description_str("Image {}, Grey ({}), White ({})".format(
      imagename, str(truth_grey_arr.shape), str(truth_white_arr.shape)) )
  t.refresh()
  t.write("", end=' ')

  # Generate background mask
  truth_back_arr = np.logical_not(np.logical_or(truth_grey_arr, truth_white_arr))

  # Save background mask
  truth_back_img = img_frombytes(truth_back_arr)
  truth_back_img.save(TRUTH_DIR + imagename + '-Background.png')

  t.update()
t.close()

print('Done!')

Image NA3777-02_AB, Grey ((47177, 53784)), White ((47177, 53784)):   0%|          | 0/18 [00:24<?, ?it/s]

Image NA4077-02_AB, Grey ((46200, 73704)), White ((46200, 73704)):   6%|▌         | 1/18 [01:49<10:50, 38.25s/it]

Image NA4092-02_AB, Grey ((45178, 65736)), White ((45178, 65736)):  11%|█         | 2/18 [03:18<14:11, 53.21s/it]

Image NA4107-02_AB, Grey ((36459, 67728)), White ((36459, 67728)):  17%|█▋        | 3/18 [04:30<15:59, 63.97s/it]

Image NA4160-02_AB, Grey ((41787, 59760)), White ((41787, 59760)):  22%|██▏       | 4/18 [04:55<15:07, 64.85s/it]

...

Image NA4675-02_AB, Grey ((45548, 53784)), White ((45548, 53784)):  83%|████████▎ | 15/18 [12:20<02:11, 43.85s/it]

Image NA4691-02_AB, Grey ((41073, 53784)), White ((41073, 53784)):  89%|████████▉ | 16/18 [12:46<01:18, 39.34s/it]

Image NA4695-02_AB, Grey ((41634, 55776)), White ((41634, 55776)):  94%|█████████▍| 17/18 [13:10<00:34, 34.63s/it]

Image NA4695-02_AB, Grey ((41634, 55776)), White ((41634, 55776)): 100%|██████████| 18/18 [13:22<00:00, 31.80s/it]

Done!


In [0]:
#@title
# Convert background masks from 8-bit to 1-bit
def img_frombytes(data):
  size = data.shape[::-1]
  databytes = np.packbits(data, axis=1)
  return Image.frombytes(mode='1', size=size, data=databytes)

#NEW_TRUTH_DIR = '/content/drive/My Drive/Alzheimer\'s Research/Ground Truth Accuracy Calculation/background/'

t = tqdm(total=len(image_names))
for i, imagename in enumerate(image_names):
  # Filename for testing and ground truth images
  truth_back_img_name = imagename + "-Background.png"

  # Load images
  truth_back_arr = np.array(Image.open(TRUTH_DIR + truth_back_img_name))
  assert(truth_back_arr.shape[0] == IMAGEDIMORIG[i, 1] and truth_back_arr.shape[1] == IMAGEDIMORIG[i, 0])
  assert(truth_back_arr.dtype == np.uint8)

  # Update tqdm descriptions
  t.set_description_str("Image {}, Background ({})".format(
      imagename, str(truth_back_arr.shape)) )
  t.refresh()
  t.write("", end=' ')

  # Create new background array
  truth_back_arr_new = np.zeros_like(truth_back_arr, dtype=np.bool)
  truth_back_arr_new[truth_back_arr>=1] = True

  # Save background mask
  truth_back_img = img_frombytes(truth_back_arr_new)
  truth_back_img.save(TRUTH_DIR + imagename + '-Background.png')

  t.update()
t.close()

print('Done!')

Image NA3777-02_AB, Background ((47177, 53784)):   0%|          | 0/18 [00:17<?, ?it/s]

Image NA4077-02_AB, Background ((46200, 73704)):   6%|▌         | 1/18 [01:43<08:53, 31.41s/it]

Image NA4092-02_AB, Background ((45178, 65736)):  11%|█         | 2/18 [02:32<13:01, 48.86s/it]

Image NA4107-02_AB, Background ((36459, 67728)):  17%|█▋        | 3/18 [03:17<12:00, 48.06s/it]

Image NA4160-02_AB, Background ((41787, 59760)):  22%|██▏       | 4/18 [03:44<10:51, 46.55s/it]

...

Image NA4675-02_AB, Background ((45548, 53784)):  83%|████████▎ | 15/18 [09:37<01:37, 32.53s/it]

Image NA4691-02_AB, Background ((41073, 53784)):  89%|████████▉ | 16/18 [10:02<01:01, 30.55s/it]

Image NA4695-02_AB, Background ((41634, 55776)):  94%|█████████▍| 17/18 [10:27<00:28, 28.53s/it]

Image NA4695-02_AB, Background ((41634, 55776)): 100%|██████████| 18/18 [10:38<00:00, 27.47s/it]

Done!


#### Calculate pixel accuracy, sensitivity, specificity and precision based on TN, FP, FN, TP

In [0]:
#@title
# Stores results: grey_TN, grey_FP, grey_FN, grey_TP, white_TN, white_FP, white_FN, white_TP
results = np.zeros((len(image_names), 8), dtype='int')

t = tqdm(total=len(image_names))
for i, imagename in enumerate(image_names):
  # Filename for testing and ground truth images
  test_grey_img_name = imagename + "-Grey.png"
  test_white_img_name = imagename + "-White.png"
  truth_grey_img_name = imagename + "-Grey.png"
  truth_white_img_name = imagename + "-White.png"

  ##### Testing Grey Matter Mask #####
  # Load images
  test_img = np.array(Image.open(TEST_DIR + test_grey_img_name))
  truth_img = Image.open(TRUTH_DIR + truth_grey_img_name)
  (width, height) = (IMAGEDIMNORM[i, 0] // downscale, IMAGEDIMNORM[i, 1] // downscale)
  truth_img = truth_img.resize((width, height), Image.BILINEAR)
  truth_img = np.array(truth_img)

  # Check dimensions
  if not (test_img.shape[0] == IMAGEDIMNORM[i, 1]//downscale and test_img.shape[1] == IMAGEDIMNORM[i, 0]//downscale):
    print('\t' + imagename + ' grey dimension WRONG!!!')

  # Update tqdm descriptions
  t.set_description_str("Image {}, Test ({}), Ground Truth ({})".format(
      imagename, str(test_img.shape), str(truth_img.shape)) )
  t.refresh()
  t.write("", end=' ')

  # Clip test_img to 1
  test_img[test_img>=1] = 1

  # Calculate true and false segmentations
  width = truth_img.shape[1]
  height = truth_img.shape[0]
  grey_TP = np.multiply(test_img, truth_img).sum()
  grey_FP = np.multiply(test_img, np.logical_not(truth_img)).sum()
  grey_FN = np.multiply(np.logical_not(test_img), truth_img).sum()
  grey_TN = np.multiply(np.logical_not(test_img), np.logical_not(truth_img)).sum()
  
  # Check results
  grey_total = width*height
  if not (grey_total == grey_TP+grey_FP+grey_FN+grey_TN):
    print('\t' + imagename + ' grey matter results WRONG!!!')

  ##### Testing White Matter Mask #####
  # Load images
  test_img = np.array(Image.open(TEST_DIR + test_white_img_name))
  truth_img = Image.open(TRUTH_DIR + truth_white_img_name)
  (width, height) = (IMAGEDIMNORM[i, 0] // downscale, IMAGEDIMNORM[i, 1] // downscale)
  truth_img = truth_img.resize((width, height), Image.BILINEAR)
  truth_img = np.array(truth_img)

  # Check dimensions
  if not (test_img.shape[0] == IMAGEDIMNORM[i, 1]//downscale and test_img.shape[1] == IMAGEDIMNORM[i, 0]//downscale):
    print('\t' + imagename + ' white dimension WRONG!!!')

  # Clip test_img to 1
  test_img[test_img>=1] = 1

  # Calculate true and false segmentations
  width = truth_img.shape[1]
  height = truth_img.shape[0]
  white_TP = np.multiply(test_img, truth_img).sum()
  white_FP = np.multiply(test_img, np.logical_not(truth_img)).sum()
  white_FN = np.multiply(np.logical_not(test_img), truth_img).sum()
  white_TN = np.multiply(np.logical_not(test_img), np.logical_not(truth_img)).sum()

  # Check results
  white_total = width*height
  if not (white_total == white_TN+white_FP+white_FN+white_TP):
    print('\t' + imagename + ' white matter results WRONG!!!')

  # Save results: grey_TN, grey_FP, grey_FN, grey_TP, white_TN, white_FP, white_FN, white_TP
  results[i, :] = [grey_TN, grey_FP, grey_FN, grey_TP, white_TN, white_FP, white_FN, white_TP]

  t.update()
t.close()

# Generate results file
with open(TEST_DIR + SAVE_FILENAME, "w") as f:
  f.write('Image Name,Grey_TN (px),Grey_FP (px),Grey_FN (px),Grey_TP (px),' +
          'White_TN (px),White_FP (px),White_FN (px),White_TP (px),' + 
          'G_Accuracy (%),G_Misclassification Rate (%),G_Sensitivity (%),' +
          'G_Specificity (%),G_Precision (%),G_Prevalence (%),' + 
          'W_Accuracy (%),W_Misclassification Rate (%),W_Sensitivity (%),' +
          'W_Specificity (%),W_Precision (%),W_Prevalence (%)\n') 
  accu_result = np.zeros((len(image_names), 12))
  for i, imagename in enumerate(image_names):
    total = results[i, 0]+results[i, 1]+results[i, 2]+results[i, 3]
    accu_result[i, 0]  = (results[i, 0]+results[i, 3])/total*100
    accu_result[i, 1]  = (results[i, 1]+results[i, 2])/total*100
    accu_result[i, 2]  = (results[i, 3]/(results[i, 2]+results[i, 3]))*100
    accu_result[i, 3]  = (results[i, 0]/(results[i, 0]+results[i, 1]))*100
    accu_result[i, 4]  = (results[i, 3]/(results[i, 1]+results[i, 3]))*100
    accu_result[i, 5]  = (results[i, 2]+results[i, 3])/total*100
    total = results[i, 4]+results[i, 5]+results[i, 6]+results[i, 7]
    accu_result[i, 6]  = (results[i, 4]+results[i, 7])/total*100
    accu_result[i, 7]  = (results[i, 5]+results[i, 6])/total*100
    accu_result[i, 8]  = (results[i, 7]/(results[i, 6]+results[i, 7]))*100
    accu_result[i, 9]  = (results[i, 4]/(results[i, 4]+results[i, 5]))*100
    accu_result[i, 10] = (results[i, 7]/(results[i, 5]+results[i, 7]))*100
    accu_result[i, 11] = (results[i, 6]+results[i, 7])/total*100
    f.write('%s,%d,%d,%d,%d,%d,%d,%d,%d,%.2f,%.2f,%.2f,%.2f,%.2f,%.2f,%.2f,%.2f,%.2f,%.2f,%.2f,%.2f\n' % 
        (imagename, results[i, 0], results[i, 1], results[i, 2], results[i, 3], 
            results[i, 4], results[i, 5], results[i, 6], results[i, 7], 
            accu_result[i, 0], accu_result[i, 1], accu_result[i, 2], 
            accu_result[i, 3], accu_result[i, 4], accu_result[i, 5], 
            accu_result[i, 6], accu_result[i, 7], accu_result[i, 8], 
            accu_result[i, 9], accu_result[i, 10], accu_result[i, 11]))
  f.write('-' * 230 + '\n')
  f.write('Average,%d,%d,%d,%d,%d,%d,%d,%d,%.2f,%.2f,%.2f,%.2f,%.2f,%.2f,%.2f,%.2f,%.2f,%.2f,%.2f,%.2f\n' % 
          (np.mean(results[:, 0]), np.mean(results[:, 1]), np.mean(results[:, 2]), np.mean(results[:, 3]), 
              np.mean(results[:, 4]), np.mean(results[:, 5]), np.mean(results[:, 6]), np.mean(results[:, 7]), 
              np.mean(accu_result[:, 0]), np.mean(accu_result[:, 1]), np.mean(accu_result[:, 2]), 
              np.mean(accu_result[:, 3]), np.mean(accu_result[:, 4]), np.mean(accu_result[:, 5]), 
              np.mean(accu_result[:, 6]), np.mean(accu_result[:, 7]), np.mean(accu_result[:, 8]), 
              np.mean(accu_result[:, 9]), np.mean(accu_result[:, 10]), np.mean(accu_result[:, 11])))

# Generate results file for LaTeX
with open(TEST_DIR + SAVE_FILENAME_LATEX, "w") as f:
  # Grey Matter
  f.write('Image Name,Grey_TN (px),Grey_FP (px),Grey_FN (px),Grey_TP (px),' +
          'G_Accuracy (%),G_Sensitivity (%),G_Specificity (%),G_Precision (%)\n')
  accu_result = np.zeros((len(image_names), 4))
  for i, imagename in enumerate(image_names):
    total = results[i, 0]+results[i, 1]+results[i, 2]+results[i, 3]
    accu_result[i, 0]  = (results[i, 0]+results[i, 3])/total*100
    accu_result[i, 1]  = (results[i, 3]/(results[i, 2]+results[i, 3]))*100
    accu_result[i, 2]  = (results[i, 0]/(results[i, 0]+results[i, 1]))*100
    accu_result[i, 3]  = (results[i, 3]/(results[i, 1]+results[i, 3]))*100
    f.write('%s & %d & %d & %d & %d & %.2f\\%% & %.2f\\%% & %.2f\\%% & %.2f\\%%\\\\\n\\hline\n' % 
        (imagename.split('-')[0], results[i, 0], results[i, 1], results[i, 2], results[i, 3], 
            accu_result[i, 0], accu_result[i, 1], accu_result[i, 2], accu_result[i, 3]))
  f.write('Average & %d & %d & %d & %d & %.2f\\%% & %.2f\\%% & %.2f\\%% & %.2f\\%%\\\\\n\n' % 
          (np.mean(results[:, 0]), np.mean(results[:, 1]), np.mean(results[:, 2]), np.mean(results[:, 3]), 
              np.mean(accu_result[:, 0]), np.mean(accu_result[:, 1]), 
              np.mean(accu_result[:, 2]), np.mean(accu_result[:, 3])))
  
  # White Matter
  f.write('Image Name,White_TN (px),White_FP (px),White_FN (px),White_TP (px),' +
          'W_Accuracy (%),W_Sensitivity (%),W_Specificity (%),W_Precision (%)\n')
  accu_result = np.zeros((len(image_names), 4))
  for i, imagename in enumerate(image_names):
    total = results[i, 4]+results[i, 5]+results[i, 6]+results[i, 7] 
    accu_result[i, 0]  = (results[i, 4]+results[i, 7])/total*100
    accu_result[i, 1]  = (results[i, 7]/(results[i, 6]+results[i, 7]))*100
    accu_result[i, 2]  = (results[i, 4]/(results[i, 4]+results[i, 5]))*100
    accu_result[i, 3] = (results[i, 7]/(results[i, 5]+results[i, 7]))*100
    f.write('%s & %d & %d & %d & %d & %.2f\\%% & %.2f\\%% & %.2f\\%% & %.2f\\%%\\\\\n\\hline\n' % 
        (imagename.split('-')[0], results[i, 4], results[i, 5], results[i, 6], results[i, 7], 
            accu_result[i, 0], accu_result[i, 1], accu_result[i, 2], accu_result[i, 3]))
  f.write('Average & %d & %d & %d & %d & %.2f\\%% & %.2f\\%% & %.2f\\%% & %.2f\\%%\\\\\n\n' % 
          (np.mean(results[:, 4]), np.mean(results[:, 5]), np.mean(results[:, 6]), np.mean(results[:, 7]), 
              np.mean(accu_result[:, 0]), np.mean(accu_result[:, 1]), 
              np.mean(accu_result[:, 2]), np.mean(accu_result[:, 3])))
print('Done!')

Image NA3777-02_AB, Test ((372, 432)), Ground Truth ((372, 432)):   0%|          | 0/18 [00:05<?, ?it/s]

Image NA4077-02_AB, Test ((372, 576)), Ground Truth ((372, 576)):   6%|▌         | 1/18 [00:13<02:32,  8.96s/it]

Image NA4092-02_AB, Test ((360, 516)), Ground Truth ((360, 516)):  11%|█         | 2/18 [00:23<02:28,  9.25s/it]

Image NA4107-02_AB, Test ((288, 540)), Ground Truth ((288, 540)):  17%|█▋        | 3/18 [00:30<02:15,  9.01s/it]

Image NA4160-02_AB, Test ((336, 468)), Ground Truth ((336, 468)):  22%|██▏       | 4/18 [00:37<01:56,  8.36s/it]

...

Image NA4675-02_AB, Test ((360, 432)), Ground Truth ((360, 432)):  83%|████████▎ | 15/18 [01:55<00:19,  6.60s/it]

Image NA4691-02_AB, Test ((324, 432)), Ground Truth ((324, 432)):  89%|████████▉ | 16/18 [02:01<00:13,  6.65s/it]

Image NA4695-02_AB, Test ((336, 444)), Ground Truth ((336, 444)):  94%|█████████▍| 17/18 [02:08<00:06,  6.47s/it]

Image NA4695-02_AB, Test ((336, 444)), Ground Truth ((336, 444)): 100%|██████████| 18/18 [02:11<00:00,  6.45s/it]

Done!


#### Calculate IoU and F1 score

In [0]:
#@title
SAVE_FILENAME = 'IoUResults.csv' # Under TEST_DIR
SAVE_FILENAME_LATEX = 'IoUResults.txt' # Under TEST_DIR

# Stores results: back_TN, back_FP, back_FN, back_TP, grey_TN, grey_FP, grey_FN, grey_TP, white_TN, white_FP, white_FN, white_TP
conf_mat = np.zeros((len(image_names), 12), dtype='int')
# Stores accuracy results: back_IoU, back_F1, grey_IoU, grey_F1, white_IoU, white_F1
results = np.zeros((len(image_names), 6))
# Stores accuracy results: back_IoU_conf, back_F1_conf, grey_IoU_conf, grey_F1_conf, white_IoU_conf, white_F1_conf
results_conf = np.zeros((len(image_names), 6))

t = tqdm(total=len(image_names))
for i, imagename in enumerate(image_names):
  # Filename for testing and ground truth images
  test_grey_img_name = imagename + "-Grey.png"
  test_white_img_name = imagename + "-White.png"
  test_back_img_name = imagename + "-Background.png"
  truth_grey_img_name = imagename + "-Grey.png"
  truth_white_img_name = imagename + "-White.png"
  truth_back_img_name = imagename + "-Background.png"

  ##### Testing Background Mask #####
  # Load images
  test_img = np.array(Image.open(TEST_DIR + test_back_img_name))
  truth_img = Image.open(TRUTH_DIR + truth_back_img_name)
  (width, height) = (IMAGEDIMNORM[i, 0] // downscale, IMAGEDIMNORM[i, 1] // downscale)
  truth_img = truth_img.resize((width, height), Image.BILINEAR)
  truth_img = np.array(truth_img)

  # Check dimensions and dtype
  if not (test_img.shape[0] == IMAGEDIMNORM[i, 1]//downscale and test_img.shape[1] == IMAGEDIMNORM[i, 0]//downscale):
    print('\t' + imagename + ' background dimension WRONG!!!')
  assert(truth_img.dtype == np.bool)

  # Update tqdm descriptions
  t.set_description_str("Image {}, Test ({}), Ground Truth ({})".format(
      imagename, str(test_img.shape), str(truth_img.shape)) )
  t.refresh()
  t.write("", end=' ')

  # Clip test_img to 1
  test_img[test_img>=1] = 1

  # Calculate true and false segmentations
  width = truth_img.shape[1]
  height = truth_img.shape[0]
  back_TP = np.multiply(test_img, truth_img).sum()
  back_FP = np.multiply(test_img, np.logical_not(truth_img)).sum()
  back_FN = np.multiply(np.logical_not(test_img), truth_img).sum()
  back_TN = np.multiply(np.logical_not(test_img), np.logical_not(truth_img)).sum()

  # Check results
  back_total = width*height
  if not (back_total == back_TP+back_FP+back_FN+back_TN):
    print('\t' + imagename + ' background results WRONG!!!')

  # Calculate IoU and F1 score
  back_IoU = np.logical_and(test_img, truth_img).sum() / np.logical_or(test_img, truth_img).sum()
  back_F1 = np.logical_and(test_img, truth_img).sum() * 2 / (test_img.sum() + truth_img.sum())

  # Calculate IoU and F1 score using TP FP FN
  back_IoU_conf = back_TP / (back_TP+back_FP+back_FN)
  back_F1_conf = 2*back_TP / (2*back_TP+back_FP+back_FN)

  ##### Testing Grey Matter Mask #####
  # Load images
  test_img = np.array(Image.open(TEST_DIR + test_grey_img_name))
  truth_img = Image.open(TRUTH_DIR + truth_grey_img_name)
  (width, height) = (IMAGEDIMNORM[i, 0] // downscale, IMAGEDIMNORM[i, 1] // downscale)
  truth_img = truth_img.resize((width, height), Image.BILINEAR)
  truth_img = np.array(truth_img)

  # Check dimensions and dtype
  if not (test_img.shape[0] == IMAGEDIMNORM[i, 1]//downscale and test_img.shape[1] == IMAGEDIMNORM[i, 0]//downscale):
    print('\t' + imagename + ' grey dimension WRONG!!!')
  assert(truth_img.dtype == np.bool)

  # Clip test_img to 1
  test_img[test_img>=1] = 1

  # Calculate true and false segmentations
  width = truth_img.shape[1]
  height = truth_img.shape[0]
  grey_TP = np.multiply(test_img, truth_img).sum()
  grey_FP = np.multiply(test_img, np.logical_not(truth_img)).sum()
  grey_FN = np.multiply(np.logical_not(test_img), truth_img).sum()
  grey_TN = np.multiply(np.logical_not(test_img), np.logical_not(truth_img)).sum()
  
  # Check results
  grey_total = width*height
  if not (grey_total == grey_TP+grey_FP+grey_FN+grey_TN):
    print('\t' + imagename + ' grey matter results WRONG!!!')

  # Calculate IoU and F1 score
  grey_IoU = np.logical_and(test_img, truth_img).sum() / np.logical_or(test_img, truth_img).sum()
  grey_F1 = np.logical_and(test_img, truth_img).sum() * 2 / (test_img.sum() + truth_img.sum())

  # Calculate IoU and F1 score using TP FP FN
  grey_IoU_conf = grey_TP / (grey_TP+grey_FP+grey_FN)
  grey_F1_conf = 2*grey_TP / (2*grey_TP+grey_FP+grey_FN)

  ##### Testing White Matter Mask #####
  # Load images
  test_img = np.array(Image.open(TEST_DIR + test_white_img_name))
  truth_img = Image.open(TRUTH_DIR + truth_white_img_name)
  (width, height) = (IMAGEDIMNORM[i, 0] // downscale, IMAGEDIMNORM[i, 1] // downscale)
  truth_img = truth_img.resize((width, height), Image.BILINEAR)
  truth_img = np.array(truth_img)

  # Check dimensions and dtype
  if not (test_img.shape[0] == IMAGEDIMNORM[i, 1]//downscale and test_img.shape[1] == IMAGEDIMNORM[i, 0]//downscale):
    print('\t' + imagename + ' white dimension WRONG!!!')
  assert(truth_img.dtype == np.bool)

  # Clip test_img to 1
  test_img[test_img>=1] = 1

  # Calculate true and false segmentations
  width = truth_img.shape[1]
  height = truth_img.shape[0]
  white_TP = np.multiply(test_img, truth_img).sum()
  white_FP = np.multiply(test_img, np.logical_not(truth_img)).sum()
  white_FN = np.multiply(np.logical_not(test_img), truth_img).sum()
  white_TN = np.multiply(np.logical_not(test_img), np.logical_not(truth_img)).sum()

  # Check results
  white_total = width*height
  if not (white_total == white_TN+white_FP+white_FN+white_TP):
    print('\t' + imagename + ' white matter results WRONG!!!')

  # Calculate IoU and F1 score
  white_IoU = np.logical_and(test_img, truth_img).sum() / np.logical_or(test_img, truth_img).sum()
  white_F1 = np.logical_and(test_img, truth_img).sum() * 2 / (test_img.sum() + truth_img.sum())

  # Calculate IoU and F1 score using TP FP FN
  white_IoU_conf = white_TP / (white_TP+white_FP+white_FN)
  white_F1_conf = 2*white_TP / (2*white_TP+white_FP+white_FN)

  # Save results: back_TN, back_FP, back_FN, back_TP, grey_TN, grey_FP, grey_FN, grey_TP, white_TN, white_FP, white_FN, white_TP
  conf_mat[i, :] = [back_TN, back_FP, back_FN, back_TP, grey_TN, grey_FP, grey_FN, grey_TP, white_TN, white_FP, white_FN, white_TP]
  # Stores accuracy results: back_IoU, back_F1, grey_IoU, grey_F1, white_IoU, white_F1
  results[i, :] = [back_IoU, back_F1, grey_IoU, grey_F1, white_IoU, white_F1]
  # Stores accuracy results: back_IoU_conf, back_F1_conf, grey_IoU_conf, grey_F1_conf, white_IoU_conf, white_F1_conf
  results_conf[i, :] = [back_IoU_conf, back_F1_conf, grey_IoU_conf, grey_F1_conf, white_IoU_conf, white_F1_conf]

  t.update()
t.close()

################################################################################
# Multiply results and results_conf by 100 (%)
results *= 100
results_conf *= 100

# Average results: Avg_IoU (%),Avg_IoU_conf (%),Avg_F1 (%),Avg_F1_conf (%)
avg_result = np.zeros((len(image_names), 4))
avg_result[:, 0] = np.mean(results[:, 0:6:2], axis=1)
avg_result[:, 1] = np.mean(results_conf[:, 0:6:2], axis=1)
avg_result[:, 2] = np.mean(results[:, 1:6:2], axis=1)
avg_result[:, 3] = np.mean(results_conf[:, 1:6:2], axis=1)

# Generate results file
with open(TEST_DIR + SAVE_FILENAME, "w") as f:
  f.write('Image Name,Back_TN (px),Back_FP (px),Back_FN (px),Back_TP (px),' +
          'Grey_TN (px),Grey_FP (px),Grey_FN (px),Grey_TP (px),' +
          'White_TN (px),White_FP (px),White_FN (px),White_TP (px),' + 
          'Back_IoU (%),Back_IoU_conf (%),Back_F1 (%),Back_F1_conf (%),' +
          'Grey_IoU (%),Grey_IoU_conf (%),Grey_F1 (%),Grey_F1_conf (%),' +
          'White_IoU (%),White_IoU_conf (%),White_F1 (%),White_F1_conf (%),' +
          'Avg_IoU (%),Avg_IoU_conf (%),Avg_F1 (%),Avg_F1_conf (%)\n') 

  for i, imagename in enumerate(image_names):
    f.write('%s,%d,%d,%d,%d,%d,%d,%d,%d,%d,%d,%d,%d,%.2f,%.2f,%.2f,%.2f,%.2f,%.2f,%.2f,%.2f,%.2f,%.2f,%.2f,%.2f,%.2f,%.2f,%.2f,%.2f\n' % 
        (imagename, conf_mat[i, 0], conf_mat[i, 1], conf_mat[i, 2], conf_mat[i, 3], 
            conf_mat[i, 4], conf_mat[i, 5], conf_mat[i, 6], conf_mat[i, 7], 
            conf_mat[i, 8], conf_mat[i, 9], conf_mat[i, 10], conf_mat[i, 11], 
            results[i, 0], results_conf[i, 0], results[i, 1], results_conf[i, 1], 
            results[i, 2], results_conf[i, 2], results[i, 3], results_conf[i, 3], 
            results[i, 4], results_conf[i, 4], results[i, 5], results_conf[i, 5],
            avg_result[i, 0], avg_result[i, 1], avg_result[i, 2], avg_result[i, 3])) 
  f.write('-' * 230 + '\n')
  f.write('Average,%d,%d,%d,%d,%d,%d,%d,%d,%d,%d,%d,%d,%.2f,%.2f,%.2f,%.2f,%.2f,%.2f,%.2f,%.2f,%.2f,%.2f,%.2f,%.2f,%.2f,%.2f,%.2f,%.2f\n' % 
          (np.mean(conf_mat[:, 0]), np.mean(conf_mat[:, 1]), np.mean(conf_mat[:, 2]), np.mean(conf_mat[:, 3]), 
              np.mean(conf_mat[:, 4]), np.mean(conf_mat[:, 5]), np.mean(conf_mat[:, 6]), np.mean(conf_mat[:, 7]), 
              np.mean(conf_mat[:, 8]), np.mean(conf_mat[:, 9]), np.mean(conf_mat[:, 10]), np.mean(conf_mat[:, 11]), 
              np.mean(results[:, 0]), np.mean(results_conf[:, 0]), np.mean(results[:, 1]), np.mean(results_conf[:, 1]), 
              np.mean(results[:, 2]), np.mean(results_conf[:, 2]), np.mean(results[:, 3]), np.mean(results_conf[:, 3]), 
              np.mean(results[:, 4]), np.mean(results_conf[:, 4]), np.mean(results[:, 5]), np.mean(results_conf[:, 5]),
              np.mean(avg_result[:, 0]), np.mean(avg_result[:, 1]), np.mean(avg_result[:, 2]), np.mean(avg_result[:, 3])))

# Generate results file for LaTeX
with open(TEST_DIR + SAVE_FILENAME_LATEX, "w") as f:
  f.write('Image Name,Back_IoU (%),Back_F1 (%),Grey_IoU (%),Grey_F1 (%),White_IoU (%),White_F1 (%),Avg_IoU (%),Avg_F1 (%)\n')
  for i, imagename in enumerate(image_names):
    f.write('%s & %.2f\\%% & %.2f\\%% & %.2f\\%% & %.2f\\%% & %.2f\\%% & %.2f\\%% & %.2f\\%% & %.2f\\%%\\\\\n\\hline\n' % 
        (imagename.split('-')[0], results[i, 0], results[i, 1], results[i, 2], 
            results[i, 3], results[i, 4], results[i, 5], 
            avg_result[i, 0], avg_result[i, 2]))
  f.write('Average & %.2f\\%% & %.2f\\%% & %.2f\\%% & %.2f\\%% & %.2f\\%% & %.2f\\%% & %.2f\\%% & %.2f\\%%\\\\\n\n' % 
          (np.mean(results[:, 0]), np.mean(results[:, 1]), np.mean(results[:, 2]), 
           np.mean(results[:, 3]), np.mean(results[:, 4]), np.mean(results[:, 5]),
              np.mean(avg_result[:, 0]), np.mean(avg_result[:, 2])))
print('Done!')

Image NA3777-02_AB, Test ((372, 432)), Ground Truth ((372, 432)):   0%|          | 0/18 [00:10<?, ?it/s]

Image NA4077-02_AB, Test ((372, 576)), Ground Truth ((372, 576)):   6%|▌         | 1/18 [00:26<05:23, 19.02s/it]

Image NA4092-02_AB, Test ((360, 516)), Ground Truth ((360, 516)):  11%|█         | 2/18 [00:43<05:04, 19.03s/it]

Image NA4107-02_AB, Test ((288, 540)), Ground Truth ((288, 540)):  17%|█▋        | 3/18 [00:58<04:32, 18.17s/it]

Image NA4160-02_AB, Test ((336, 468)), Ground Truth ((336, 468)):  22%|██▏       | 4/18 [01:11<03:50, 16.46s/it]

...

Image NA4299-02_AB, Test ((348, 456)), Ground Truth ((348, 456)):  39%|███▉      | 7/18 [01:51<02:37, 14.28s/it]

Image NA4391-02_AB, Test ((372, 396)), Ground Truth ((372, 396)):  44%|████▍     | 8/18 [02:03<02:21, 14.17s/it]

Image NA4450-02_AB, Test ((372, 456)), Ground Truth ((372, 456)):  50%|█████     | 9/18 [02:15<02:01, 13.50s/it]

Image NA4463-02_AB, Test ((360, 432)), Ground Truth ((360, 432)):  56%|█████▌    | 10/18 [02:29<01:49, 13.63s/it]

Image NA4471-02_AB, Test ((372, 564)), Ground Truth ((372, 564)):  61%|██████    | 11/18 [02:43<01:32, 13.27s/it]